In [1]:
import os

In [2]:
%pwd

'd:\\ML Projects\\bird_type_classification_project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\ML Projects\\bird_type_classification_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    validation_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_acceptable_classes: list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:

        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "train")
        validation_data = os.path.join(self.config.data_ingestion.unzip_dir, "valid")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            validation_data = Path(validation_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_acceptable_classes=params.ACCEPTABLE_CLASSES
        )

        return training_config

[20-Nov-23 00:27:45: INFO: utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[20-Nov-23 00:27:45: INFO: utils: NumExpr defaulting to 8 threads.]


In [7]:
import os
import tensorflow as tf
import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath.__str__(),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [10]:
import os
import tensorflow as tf
import time
from keras.preprocessing.image import ImageDataGenerator
from cnnClassifier.utils.common import generate_dataframe

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        return tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        train_df = generate_dataframe(
            self.config.training_data, self.config.params_acceptable_classes
        )

        valid_df = generate_dataframe(
            self.config.validation_data, self.config.params_acceptable_classes
        )

        generator_kwargs = dict(
            rescale = 1./255,
        )

        dataflow_kwargs = dict(
            x_col = 'filepath',
            y_col = 'label',
            target_size = self.config.params_image_size[:-1],
            color_mode = 'rgb',
            batch_size = self.config.params_batch_size
        )

        val_gen = ImageDataGenerator(**generator_kwargs)

        if self.config.params_is_augmentation:
            tr_gen = ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **generator_kwargs)
        else:
            tr_gen = ImageDataGenerator(**generator_kwargs)
            
        print('Valid data flow')
        valid_gen = val_gen.flow_from_dataframe(
            dataframe=valid_df,
            shuffle=False,
            **dataflow_kwargs
        )
        
        print('Train data flow')
        train_gen = tr_gen.flow_from_dataframe(
            dataframe=train_df,
            shuffle=True,
            **dataflow_kwargs
        )
        return (train_gen, valid_gen)
    
    def train(self, callback_list:list):

        train_gen, valid_gen = self.train_valid_generator()
        model = self.get_base_model()

        model.fit(
            x =  train_gen,
            verbose = 1,
            shuffle = False,
            epochs= self.config.params_epochs,
            validation_data = valid_gen,
            callbacks = callback_list,
            use_multiprocessing=True,
            workers=tf.data.experimental.AUTOTUNE
        )

        self.save_model(
            path = self.config.trained_model_path,
            model=model
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [11]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    print(callback_list)

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[20-Nov-23 00:56:05: INFO: common: config\config.yaml loaded successfully.]
[20-Nov-23 00:56:05: INFO: common: params.yaml loaded successfully.]
[20-Nov-23 00:56:05: INFO: common: Created directory at: artifacts]
[20-Nov-23 00:56:05: INFO: common: Created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[20-Nov-23 00:56:05: INFO: common: Created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[<keras.src.callbacks.TensorBoard object at 0x00000275F02CE690>, <keras.src.callbacks.ModelCheckpoint object at 0x00000275F48533D0>]
[20-Nov-23 00:56:05: INFO: common: Created directory at: artifacts\training]
Valid data flow
Found 25 validated image filenames belonging to 5 classes.
Train data flow
Found 768 validated image filenames belonging to 5 classes.
<class 'keras.src.preprocessing.image.DataFrameIterator'>
Valid data flow
Found 25 validated image filenames belonging to 5 classes.
Train data flow
Found 768 validated image filenames belonging to 5 classes.
<class 'ke

d:\ML Projects\mlvenv\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


24/24 [==============================] - 30s 960ms/step - loss: 9.6694 - accuracy: 0.1992 - val_loss: 11.5670 - val_accuracy: 0.2000
Epoch 2/5
24/24 [==============================] - 21s 883ms/step - loss: 8.6327 - accuracy: 0.2109 - val_loss: 9.1187 - val_accuracy: 0.2000
Epoch 3/5
24/24 [==============================] - 22s 923ms/step - loss: 8.1944 - accuracy: 0.2044 - val_loss: 8.2629 - val_accuracy: 0.2000
Epoch 4/5
24/24 [==============================] - 22s 904ms/step - loss: 7.8527 - accuracy: 0.2096 - val_loss: 8.1000 - val_accuracy: 0.2000
Epoch 5/5
24/24 [==============================] - 21s 881ms/step - loss: 7.4705 - accuracy: 0.1810 - val_loss: 7.2925 - val_accuracy: 0.2000
